[Cross validation 사용하는 이유](https://towardsdatascience.com/5-reasons-why-you-should-use-cross-validation-in-your-data-science-project-8163311a1e79)

[Tensor](https://m.blog.naver.com/PostView.nhn?blogId=complusblog&logNo=221237818389&proxyReferer=https%3A%2F%2Fwww.google.com%2F)

[What’s the difference between a matrix and a tensor?](https://medium.com/@quantumsteinke/whats-the-difference-between-a-matrix-and-a-tensor-4505fbdc576c)

In [0]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, tset_targets) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 0us/step


In [0]:
print(f'Training data : {train_data.shape}')
print(f'Test data : {test_data.shape}')
print(f'Training sample : {train_data[0]}')
print(f'Training target sample : {train_targets[0]}')

Training data : (404, 13)
Test data : (102, 13)
Training sample : [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
Training target sample : 15.2


In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [0]:
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  # input_shape = 13개 한 줄씩 읽어온다.
  
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  # identity activation == regression 이라서 값을 그대로 가져와야함
  
  model.compile(optimizer='rmsprop',
               loss='mse',
               metrics=['mae'])
  return model


In [0]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

# cross validation 구현
# data의 수가 적어서 cross validation 사용함
# 
# shuffle하고 for loop 진행하면 더 성능이 좋아질 가능성이 있다.
for i in range(k):
  print(f'Processing fold # {i}')
  val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
  
  partial_train_data = np.concatenate(  
  [train_data[:i * num_val_samples],
  train_data[(i+1) * num_val_samples:]],
  axis=0)  # 행 방향으로 붙임
  
  # 첫 iter에서 : traindat[:0] == None, traindat[100:]

  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i+1) * num_val_samples:]],
  axis=0)
  
  model = build_model()
  model.fit(partial_train_data, 
            partial_train_targets,
            epochs=num_epochs,
            batch_size=1,
            verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

W0708 13:18:31.794591 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 13:18:31.846650 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 13:18:31.854669 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 13:18:31.901571 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Processing fold # 0


W0708 13:18:32.102095 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0708 13:18:32.117167 139803991037824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Processing fold # 1
Processing fold # 2
Processing fold # 3


In [0]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')
# 평균으로 모델의 성능을 알 수 있다.

all_scores : [2.0581326413862775, 2.2001309324019025, 2.8750618802438868, 2.410775157484678]
mean all scores : 2.386025152879186


In [0]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, tset_targets)

102/102 [==============================] - 0s 735us/step


In [0]:
test_mse_score, test_mae_score
# 정답과 예측값의 실제 차이를 확인하기 위해서 mae사용

(20.850413602941178, 3.0243620124517703)